In [1]:
from ogb.lsc import MAG240MDataset
dataset = MAG240MDataset(root = "/home/lc/Developer/mag240m")

In [2]:
import csv
import numpy as np
import pandas as pd
import json
import math
from tqdm import tqdm
import pyarrow as pa

In [3]:
all_paper_year = dataset.all_paper_year.astype(np.int16)

In [4]:
all_paper_label = dataset.all_paper_label.astype(np.int16)

/tmp/ipykernel_1453900/165144024.py:1: RuntimeWarning: invalid value encountered in cast
  all_paper_label = dataset.all_paper_label.astype(np.int16)


In [5]:
all_paper_feat = dataset.all_paper_feat

In [11]:
type(all_paper_feat)

numpy.ndarray

In [12]:
all_paper_feat.dtype

dtype('float16')

In [6]:
all_paper_feat_list = []
for i in tqdm(range(len(all_paper_feat))):
    all_paper_feat_list.append(all_paper_feat[i])

100%|██████████| 121751666/121751666 [00:54<00:00, 2234917.99it/s]


In [7]:
df = pd.DataFrame({'year': all_paper_year, 'label': all_paper_label, 'feat': all_paper_feat_list})

In [8]:
df.index.name = 'id'

In [9]:
df

,year,label,feat
id,,,
0,2004,0,"[-0.2715, -0.1475, 0.1984, -0.1376, 0.557, -0...."
1,2004,0,"[-0.1743, 0.3687, 0.06195, 0.4053, 0.706, 0.10..."
2,2008,0,"[0.558, -0.5654, 0.3413, 1.242, 0.3665, 0.4607..."
3,2005,0,"[0.2205, -0.2489, 0.2007, -0.1948, 0.4016, 0.3..."
4,2005,0,"[-0.8223, -0.234, 0.3374, 0.5225, 1.123, -0.09..."
...,...,...,...
121751661,2018,0,"[-0.5054, -0.272, -0.9053, -0.3267, 0.04315, 0..."
121751662,2018,127,"[0.4346, 0.0758, -0.653, -0.0578, 0.06137, -0...."
121751663,2019,0,"[0.2644, 0.1191, 0.413, 0.773, -0.4077, -0.282..."


In [10]:
chunk_size = 100000
num_chunks = math.ceil(len(df) / chunk_size)
for i in tqdm(range(num_chunks)):
    chunk = df.iloc[i * chunk_size:(i + 1) * chunk_size].copy(deep=True)
    chunk['feat'] = chunk['feat'].apply(lambda x: str(x.tolist()).replace(' ', ''))
    chunk.to_parquet('papers.parquet', engine='fastparquet', compression='snappy', index=True, append=(i > 0))
    del chunk

100%|██████████| 1218/1218 [16:38:49<00:00, 49.20s/it]  


In [4]:
feat = dataset.paper_feat
feat_f32 = np.zeros_like(feat, dtype=np.float32)
for i in tqdm(range(len(feat))):
    feat_f32[i] = feat[i].astype(np.float32)

100%|██████████| 121751666/121751666 [19:59<00:00, 101483.90it/s]


In [5]:
np.save('/home/lc/Developer/mag240m/mag240m_kddcup2021/processed/paper/node_feat_f32.npy', feat_f32)